# Chapter 1: 基础的提示词结构

- [Lesson](#lesson)
- [Exercises](#exercises)
- [Example Playground](#example-playground)


## 设置

运行以下设置单元格以加载您的API密钥并建立`get_completion`辅助函数。

In [1]:
!pip install anthropic

# Import python's built-in regular expression library
import re
import anthropic

# Retrieve the API_KEY & MODEL_NAME variables from the IPython store
%store -r API_KEY
%store -r MODEL_NAME

client = anthropic.Anthropic(api_key=API_KEY)

def get_completion(prompt: str, system_prompt=""):
    message = client.messages.create(
        model=MODEL_NAME,
        max_tokens=2000,
        temperature=0.0,
        system=system_prompt,
        messages=[
          {"role": "user", "content": prompt}
        ]
    )
    return message.content[0].text



## 课程

Anthropic提供两个API，传统的[Text Completions API](https://docs.anthropic.com/claude/reference/complete_post)和当前的[Messages API](https://docs.anthropic.com/claude/reference/messages_post)。对于本教程，我们将专门使用Messages API。

使用Messages API调用Claude至少需要以下参数：
- `model`：您打算调用的模型的[API模型名称](https://docs.anthropic.com/claude/docs/models-overview#model-recommendations)

- `max_tokens`：在停止之前生成的最大token数量。请注意，Claude可能在达到此最大值之前停止。此参数仅指定要生成的绝对最大token数量。此外，这是一个*硬性*停止，意味着它可能导致Claude在单词中间或句子中间停止生成。

- `messages`：输入消息数组。我们的模型经过训练，可以在交替的`user`和`assistant`对话轮次中运行。创建新的`Message`时，您使用messages参数指定先前的对话轮次，然后模型生成对话中的下一个`Message`。
  - 每个输入消息必须是一个具有`role`和`content`的对象。您可以指定单个`user`角色的消息，或者可以包含多个`user`和`assistant`消息（如果是这样，它们必须交替）。第一条消息必须始终使用user`role`。

还有可选参数，例如：
- `system`：系统提示 - 更多内容见下文。
  
- `temperature`：Claude响应中的变化程度。对于这些课程和练习，我们将`temperature`设置为0。



### 示例

让我们来看一下Claude对一些格式正确的提示的回应。对于下方的每个单元格，运行该单元格（`shift+enter`），Claude的回复会显示在代码块下方。

In [3]:
# Prompt
PROMPT = "Hi Claude, 你好吗?"

# Print Claude's response
print(get_completion(PROMPT))

很高兴认识你!我作为一个人工智能助手,一直在努力学习和提升自己,希望能为人类提供有用的帮助。我会尽我所能回答你的问题,如果有什么不明白的地方也欢迎随时问我。让我们一起探讨有趣的话题吧!


In [4]:
# Prompt
PROMPT = "你能告诉我大海是什么颜色的吗?"

# Print Claude's response
print(get_completion(PROMPT))

大海的颜色是多变的,主要取决于以下几个因素:

1. 水深:浅海区域通常呈现蓝绿色,而深海区域则呈现深蓝色。

2. 天气和光线:晴天时,海面反射天空的颜色,呈现蓝色。阴天时,海面会显得灰蒙蒙的。

3. 海底地形和沉积物:海底的岩石、沙子等会影响海水的颜色。有时会出现绿色、褐色等。

4. 海藻和浮游生物:海藻和浮游生物的种类和数量也会影响海水的颜色,有时会呈现绿色或褐色。

总的来说,大海最常见的颜色是蓝色和蓝绿色。但由于各种因素的影响,大海的颜色会有很大变化,有时甚至会呈现不同的色调。这就是大海的魅力所在。


In [6]:
# Prompt
PROMPT = "科比比来恩特是哪年出生的?"

# Print Claude's response
print(get_completion(PROMPT))

科比·布莱恩特(Kobe Bryant)是1978年8月23日出生的。

他出生于美国宾夕法尼亚州费城,是一名职业篮球运动员,曾效力于NBA洛杉矶湖人队,被认为是NBA历史上最伟大的球员之一。

科比·布莱恩特在2020年1月26日不幸在一次直升机坠毁事故中去世,终年41岁。他的去世引发了全球范围内的哀悼和追忆。



现在让我们来看一些没有采用正确Messages API格式的提示。对于这些格式错误的提示，Messages API会返回错误。

首先，我们来看一个在`messages`数组中缺少`role`和`content`字段的Messages API调用示例。

In [ ]:
# Get Claude's response
response = client.messages.create(
        model=MODEL_NAME,
        max_tokens=2000,
        temperature=0.0,
        messages=[
          {"Hi Claude, how are you?"}
        ]
    )

# Print Claude's response
print(response[0].text)

下面是一个没有在 `user` 和 `assistant` 角色之间交替的提示示例。

In [9]:
# Get Claude's response
response = client.messages.create(
        model=MODEL_NAME,
        max_tokens=2000,
        temperature=0.0,
        messages=[
          {"role": "user", "content": "科比比来恩特是哪年出生的??"},
          {"role": "user", "content": "也告诉关于他的另外一些事情?"}
        ]
    )

# Print Claude's response
print(response[0].text)

TypeError: 'Message' object is not subscriptable


`user` 和 `assistant` 消息**必须交替**，并且消息**必须以 `user` 轮次开始**。您可以在提示中有多个 `user` 和 `assistant` 对（就像模拟多轮对话一样）。您还可以在最终的 `assistant` 消息中放入文字，让Claude从您停止的地方继续（更多内容将在后面的章节中介绍）。

#### 系统提示

您还可以使用**系统提示**。系统提示是一种在"用户"轮次中向Claude提出问题或任务之前**为其提供上下文、指令和指导**的方式。

从结构上讲，系统提示与 `user` 和 `assistant` 消息列表分开存在，因此属于单独的 `system` 参数（请查看notebook中[设置](#setup)部分的 `get_completion` 辅助函数的结构）。

在本教程中，无论我们在哪里使用系统提示，我们都在您的完成函数中为您提供了 `system` 字段。如果您不想使用系统提示，只需将 `SYSTEM_PROMPT` 变量设置为空字符串即可。

#### System Prompt Example

In [11]:
# System prompt
SYSTEM_PROMPT = "您的回答应该始终是一系列能够推进对话的批判性思维问题（不要为您的问题提供答案）。不要实际回答用户的问题。."

# Prompt
PROMPT = "天空是什么颜色的"

# Print Claude's response
print(get_completion(PROMPT, SYSTEM_PROMPT))

那是一个很好的问题!我们可以从几个角度来探讨天空的颜色:

- 天空的颜色是如何形成的?是什么原因导致天空呈现出蓝色或其他颜色?

- 天空的颜色会随时间和天气条件发生变化吗?为什么会有这种变化?

- 不同地区或气候条件下,天空的颜色会有什么差异?这些差异背后有什么原因吗?

- 我们如何通过观察天空的颜色来了解当地的气候和环境?这种观察有什么实际应用价值吗?

通过探讨这些问题,我们可以更深入地理解天空颜色的形成机理,以及它与自然环境的关系。您对这些问题有什么想法吗?我很乐意听听您的看法。



为什么使用系统提示词？一个**编写良好的系统提示可以通过多种方式提高Claude的性能**，比如增强Claude遵循规则和指令的能力。更多信息，请访问我们关于[如何使用系统提示](https://docs.anthropic.com/claude/docs/how-to-use-system-prompts)的文档。

现在我们将深入一些练习。如果您想在不更改上述任何内容的情况下实验课程提示，请滚动到课程notebook的底部访问[**示例实验区**](#example-playground)。

---

## 练习
- [练习 1.1 - Counting to Three](#exercise-11---counting-to-three)
- [练习 1.2 - System Prompt](#exercise-12---system-prompt)


### 练习 1.1 - 数到三
使用正确的 `user` / `assistant` 格式，编辑下面的 `PROMPT` 让Claude**数到三**。输出还会显示您的解决方案是否正确。

In [ ]:
# Prompt - this is the only field you should change
PROMPT = "[Replace this text]"

# Get Claude's response
response = get_completion(PROMPT)

# Function to grade exercise correctness
def grade_exercise(text):
    pattern = re.compile(r'^(?=.*1)(?=.*2)(?=.*3).*$', re.DOTALL)
    return bool(pattern.match(text))

# Print Claude's response and the corresponding grade
print(response)
print("\n--------------------------- GRADING ---------------------------")
print("This exercise has been correctly solved:", grade_exercise(response))

1, 2, 3.

--------------------------- GRADING ---------------------------
This exercise has been correctly solved: True



### 练习 1.2 - 系统提示

修改 `SYSTEM_PROMPT` 使Claude像3岁孩子一样回应。

In [ ]:
# System prompt - this is the only field you should change
SYSTEM_PROMPT = "[Replace this text]"

# Prompt
PROMPT = "How big is the sky?"

# Get Claude's response
response = get_completion(PROMPT, SYSTEM_PROMPT)

# Function to grade exercise correctness
def grade_exercise(text):
    return bool(re.search(r"giggles", text) or re.search(r"soo", text))

# Print Claude's response and the corresponding grade
print(response)
print("\n--------------------------- GRADING ---------------------------")
print("This exercise has been correctly solved:", grade_exercise(response))


### 恭喜！

如果您已经解决了到目前为止的所有练习，您就可以进入下一章了。祝您提示愉快！